# Implementacion de Modelos de Redes Neuronales a Grafos.

**Autor: Jorge Xavier Romero**

# Parte 1: Predicción de etiquetas de nodos

## 0. Importas generales e instalación

In [ ]:
import os
import torch

if 'IS_GRADESCOPE_ENV' not in os.environ:
#   !pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
#   !pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
#   !pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
#   !pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
  !pip install git+https://github.com/pyg-team/pytorch_geometric.git
  !pip install torch-geometric
  !pip install ogb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-cqyfqd0d
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-cqyfqd0d
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 5ff3295250715f765ac60dfd6c34e0fc7f1cd904
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.4.0-py3-none-any.whl size=960669 sha256=a0c2ce59800c83720e28070f01ec8e384742368cd18d53b88c5c6ed3645161e4
  Stored in directory: /tmp/pip-ephem-wheel-cache-fx3832qt/wheels/d3/78/eb/9e26525b948d19533f1688fb6c209cec8a0ba793d39b49ae8f
Successfully built torch_geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
L

## 1. Carga del dataset

Te recomendamos montar tu drive en colab y luego utilizar `gdown` para descargar los datos a drive o, alternativamente, descargar los datos desde [este link](https://drive.google.com/file/d/17eMe-4MFx8mmVuLgf6pcz1NVKJ9cP96u/view?usp=sharing) y luego subirlos manualmente a drive.

Para cargar el dataset debes utilizar pickle para deserializar el archivo `dataset` y luego responder las preguntas que se encuentran a continuación.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!gdown 17eMe-4MFx8mmVuLgf6pcz1NVKJ9cP96u

Downloading...
From: https://drive.google.com/uc?id=17eMe-4MFx8mmVuLgf6pcz1NVKJ9cP96u
To: /content/dataset
100% 895M/895M [00:16<00:00, 54.4MB/s]


In [ ]:
import pickle

with open('drive/MyDrive//Backup/dataset', 'rb') as file:
  data = pickle.load(file)


In [ ]:
print(data.x)
print(data.edge_index)
print(data.y)


None
None
None


In [ ]:
print(type(data))
print(data)

<class 'torch_geometric.data.data.Data'>
Data(
  num_nodes_dict={
    author=1134649,
    field_of_study=59965,
    institution=8740,
    paper=736389,
  },
  edge_index_dict={
    (author, affiliated_with, institution)=[2, 1043998],
    (author, writes, paper)=[2, 7145660],
    (paper, cites, paper)=[2, 5416271],
    (paper, has_topic, field_of_study)=[2, 7505078],
  },
  x_dict={ paper=[736389, 128] },
  node_year={ paper=[736389, 1] },
  edge_reltype={
    (author, affiliated_with, institution)=[1043998, 1],
    (author, writes, paper)=[7145660, 1],
    (paper, cites, paper)=[5416271, 1],
    (paper, has_topic, field_of_study)=[7505078, 1],
  },
  y_dict={ paper=[736389, 1] }
)


In [ ]:
# Imprimir el número de nodos y bordes para cada tipo
for node_type, num in data.num_nodes_dict.items():
    print(f"{node_type}: {num} nodos")

for edge_type, edge_index in data.edge_index_dict.items():
    print(f"{edge_type}: {len(edge_index[0])} bordes")

# Imprimir algunas características y etiquetas de 'paper'
print("Primeras 5 características de 'paper':", data.x_dict['paper'][:5])
print("Primeras 5 etiquetas de 'paper':", data.y_dict['paper'][:5])


author: 1134649 nodos
field_of_study: 59965 nodos
institution: 8740 nodos
paper: 736389 nodos
('author', 'affiliated_with', 'institution'): 1043998 bordes
('author', 'writes', 'paper'): 7145660 bordes
('paper', 'cites', 'paper'): 5416271 bordes
('paper', 'has_topic', 'field_of_study'): 7505078 bordes
Primeras 5 características de 'paper': tensor([[-9.5379e-02,  4.0758e-02, -2.1095e-01, -6.4362e-02, -2.2594e-01,
         -2.3230e-03, -5.5583e-01, -3.5291e-01,  4.0634e-02, -1.9523e-01,
         -1.2195e-01, -4.9836e-01,  5.7569e-02, -1.4867e-01, -1.5447e-01,
          2.6752e-01, -7.8571e-02,  3.0968e-01,  1.6450e-01, -1.5782e-01,
         -2.0708e-01,  2.6488e-01, -2.7965e-01, -1.4788e-01,  1.8810e-03,
          4.9859e-01,  2.6378e-01, -1.7021e-01, -1.5862e-01,  1.7871e-01,
         -9.6597e-02, -1.2366e-01, -2.5665e-02, -1.0387e-01, -1.6596e-02,
          6.8700e-04, -2.2242e-01,  5.5000e-05,  8.0517e-02,  4.9467e-01,
          1.9211e-01, -6.4611e-02,  4.5427e-02, -1.0394e-02,  3.743

In [ ]:
# Verificar que todos los índices de bordes son válidos
for edge_type, edge_index in data.edge_index_dict.items():
    src_type, _, dst_type = edge_type
    assert edge_index[0].max() < data.num_nodes_dict[src_type]
    assert edge_index[1].max() < data.num_nodes_dict[dst_type]

# Verificar que las características y las etiquetas tienen la longitud correcta
assert data.x_dict['paper'].shape[0] == data.num_nodes_dict['paper']
assert data.y_dict['paper'].shape[0] == data.num_nodes_dict['paper']


#### Preguntas

Una vez cargado el dataset debes responder las siguientes preguntas:

1. ¿De qué tipo de objeto es el dataset cargado?

R. El dataset cargado es de clase torch.geometric, el mismo que corresponde a un grafo que cuenta con 4 nodos y 4 relaciones.


2. ¿Qué tipos de nodos y aristas tiene el grafo?

R. El grafo tiene los siguientes tipos de nodos y aristas:

Tipos de nodos: 'author', 'field_of_study', 'institution', 'paper'
Tipos de aristas:
('author', 'affiliated_with', 'institution')
('author', 'writes', 'paper')
('paper', 'cites', 'paper')
('paper', 'has_topic', 'field_of_study')

3. ¿Cuántas features tiene cada nodo y arista?

R. Cada nodo del tipo 'paper' tiene 128 features.

Cada una de las aristas tiene un feature

4. ¿Cuántos nodos y aristas, de cada tipo, forman el grafo?

R. Número de nodos:
'author': 1,134,649 nodos
'field_of_study': 59,965 nodos
'institution': 8,740 nodos
'paper': 736,389 nodos

Número de aristas:
('author', 'affiliated_with', 'institution'): 1,043,998 aristas
('author', 'writes', 'paper'): 7,145,660 aristas
('paper', 'cites', 'paper'): 5,416,271 aristas
('paper', 'has_topic', 'field_of_study'): 7,505,078 aristas


## 2. Filtro de datos

In [ ]:

with open('drive/MyDrive//Backup/split', 'rb') as file:
  data = pickle.load(file)

Solamente utilizaremos los datos que relacionan papers entre sí, para luego realizar predicciones.

Crea un nuevo dataset a partir de los datos cargados que contenga:
- En x la información de los papers de `x_dict`
- En y la información de los papers de `y_dict`
- En edge_index la información de las aristas con la forma `('paper', 'cites', 'paper')`

In [ ]:
import torch
from torch_geometric.data import Data

# Obtener los datos específicos que necesitamos
x = data.x_dict['paper']
edge_index = data.edge_index_dict[('paper', 'cites', 'paper')]
y = data.y_dict['paper']


# Crear un nuevo objeto Data con los datos seleccionados
new_data = Data(x=x, y=y, edge_index=edge_index)
print(new_data)


Data(x=[736389, 128], edge_index=[2, 5416271], y=[736389, 1])


In [ ]:
# Tamaño del dataset en términos de nodos y aristas
print("Número de nodos en el nuevo dataset:", new_data.num_nodes)
print("Número de aristas en el nuevo dataset:", new_data.num_edges)

# Número de características por nodo (dimensión de 'x') y número de etiquetas (dimensión de 'y')
print("Número de características por nodo:", new_data.num_node_features)
print("Número de etiquetas:", new_data.y.size(0))

# Comparación con el dataset original
original_num_nodes = data.num_nodes_dict['paper']
original_num_edges = data.edge_index_dict[('paper', 'cites', 'paper')].size(1)

# Asumiendo que las características del nodo 'paper' y las etiquetas también se almacenan en el dataset original
original_num_node_features = data.x_dict['paper'].size(1)
original_num_labels = data.y_dict['paper'].size(0)

# Comparar
print("¿Mismo número de nodos que el dataset original?", new_data.num_nodes == original_num_nodes)
print("¿Mismo número de aristas que el dataset original?", new_data.num_edges == original_num_edges)
print("¿Mismo número de características por nodo que el dataset original?", new_data.num_node_features == original_num_node_features)
print("¿Mismo número de etiquetas que el dataset original?", new_data.y.size(0) == original_num_labels)

Número de nodos en el nuevo dataset: 736389
Número de aristas en el nuevo dataset: 5416271
Número de características por nodo: 128
Número de etiquetas: 736389
¿Mismo número de nodos que el dataset original? True
¿Mismo número de aristas que el dataset original? True
¿Mismo número de características por nodo que el dataset original? True
¿Mismo número de etiquetas que el dataset original? True


In [ ]:
# Tamaño del dataset creado
size_dataset = len(new_data)
print("Tamaño del dataset creado:", size_dataset)

# Cantidad de nodos, aristas y etiquetas en el dataset creado
num_nodes = new_data.num_nodes
num_edges = new_data.num_edges
num_labels = new_data.y.shape[0]
print("Cantidad de nodos:", num_nodes)
print("Cantidad de aristas:", num_edges)
print("Cantidad de etiquetas:", num_labels)

# Comparación con el dataset original
original_num_nodes = data.num_nodes_dict['paper']
original_num_edges = data.edge_index_dict[('paper', 'cites', 'paper')].shape[1]
original_num_labels = data.y_dict['paper'].shape[0]

if num_nodes == original_num_nodes and num_edges == original_num_edges and num_labels == original_num_labels:
    print("El dataset creado tiene la misma cantidad de datos que el dataset original.")
else:
    print("El dataset creado no tiene la misma cantidad de datos que el dataset original.")


Tamaño del dataset creado: 3
Cantidad de nodos: 736389
Cantidad de aristas: 5416271
Cantidad de etiquetas: 736389
El dataset creado tiene la misma cantidad de datos que el dataset original.


#### Preguntas

1. ¿De qué tamaño queda el dataset creado?

R. El dataset creado tiene un tamaño de 3. Este número probablemente se refiere a la cantidad de elementos en el objeto Data (es decir, x, y y edge_index).

Nos quedan 736389 nodos y 5416271 relaciones.

2. ¿Cuántos nodos, aristas y labels tiene?

Nodos: 736389. Aristas: 5416271. Labels: 736389. Caracteristicas por nodo: 128.

3. Comparando el nuevo dataset con el original, pero considerando solo los tipos en común, ¿Tenemos la misma cantidad de datos que el dataset original? ¿Por qué?

R. Tenemos menor cantidad de datos. Esto debido a que solo estamos tomando en cuenta nodos llamados paper citados por otros papers. Por lo tanto es de esperar que contenemos con una substancial menor cantidad de datos.

## 3. Data augmentation

Para aprovechar de mejor manera la información del grafo original, queremos obtener información adicional a partir de los datos que no estamos utilizando.

En primer lugar, obtén del dataset todas las aristas que relacionan los papers con sus autores. Puede que te sea de utilidad, para el paso siguiente, generar un diccionario o estructura similar que relacione a cada autor del grafo con todos los papers que escribió.

En segundo lugar, crea un nuevo dataset, que contenga toda la información del que acabas de crear en el paso 2. y agrega a el aristas entre todos los papers que tengan el mismo autor.

In [ ]:
import torch
from torch_geometric.data import Data

# Obtener aristas de papers con autores
author_paper_edges = data.edge_index_dict[('author', 'writes', 'paper')]

In [ ]:
# Crear diccionario de autores y sus papers
author_papers = {}
for i in range(author_paper_edges.size(1)):
    author_idx = author_paper_edges[0, i].item()
    paper_idx = author_paper_edges[1, i].item()
    if author_idx not in author_papers:
        author_papers[author_idx] = [paper_idx]
    else:
        author_papers[author_idx].append(paper_idx)

In [ ]:
# Imprimir los primeros 5 autores y los papers asociados
for author_idx in list(author_papers.keys())[:5]:
    papers = author_papers[author_idx]
    print(f"Autor {author_idx}: Papers {papers}")

Autor 0: Papers [19703, 289285, 311768, 402711]
Autor 1: Papers [181505, 297095, 336569]
Autor 2: Papers [14217, 14630, 15376, 15420, 20656, 27715, 37354, 62987, 78157, 86646, 91551, 103412, 129861, 148900, 178805, 190892, 194646, 261498, 301784, 357433, 432992, 455787, 489838, 519995, 540904, 599199, 600495, 614248, 617244, 647287, 689138, 705788, 722195, 729975]
Autor 3: Papers [230166, 392216, 504678, 625373]
Autor 4: Papers [240904, 393817, 499283, 538604, 621886, 709654]


In [ ]:
# Concatenar las nuevas aristas con las aristas originales
edge_index = torch.cat([edge_index, torch.tensor(same_author_edges).t()], dim=1)

# Crear el nuevo dataset con la información actualizada
new_data = Data(x=x, y=y, edge_index=edge_index)

NameError: ignored

In [ ]:
print(data.edge_index_dict.keys())


dict_keys([('author', 'affiliated_with', 'institution'), ('author', 'writes', 'paper'), ('paper', 'cites', 'paper'), ('paper', 'has_topic', 'field_of_study')])


In [ ]:
# Obtener las aristas de autor a paper
import torch
from torch_geometric.data import Data

author_paper_edge_index = data.edge_index_dict[('author', 'writes', 'paper')]


In [ ]:
# Crear un diccionario que mapea cada autor a todos los papers que escribió
author_to_papers = {}
for i in range(author_paper_edge_index.shape[1]):
    paper_node = author_paper_edge_index[0, i].item()
    author_node = author_paper_edge_index[1, i].item()
    if author_node not in author_to_papers:
        author_to_papers[author_node] = []
    author_to_papers[author_node].append(paper_node)


In [ ]:
# Iterar sobre los elementos del diccionario
for i, (author, papers) in enumerate(author_to_papers.items()):
    # Imprimir el autor y sus papers
    print(f"Autor {author}: {papers}")

    # Interrumpir el bucle después de imprimir 5 autores
    if i == 4:
        break


Autor 19703: [0, 41685, 84368, 360028, 387940]
Autor 289285: [0, 150451, 360028, 387940]
Autor 311768: [0, 150451, 360028, 387940, 1071545]
Autor 402711: [0, 360028, 387940]
Autor 181505: [1, 912752, 1045321]


In [ ]:
# Crear una lista para almacenar las nuevas aristas entre papers
new_edges = []

# Para cada autor, seleccionar cada par de papers adyacentes y agregar una arista entre ellos
for author, papers in author_to_papers.items():
    if len(papers) > 1:
        # Ordenar los papers por su índice
        sorted_papers = sorted(papers)
        # Agregar una arista entre cada par de papers adyacentes
        for i in range(len(sorted_papers) - 1):
            new_edges.append((sorted_papers[i], sorted_papers[i+1]))

# Convertir la lista de tuplas en un tensor
new_edges_tensor = torch.tensor(new_edges, dtype=torch.long).t().contiguous()

# Agregar las nuevas aristas al dataset existente
augmented_edge_index = torch.cat([new_data.edge_index, new_edges_tensor], dim=1)

# Crear un nuevo objeto Data con la información adicional
augmented_data = Data(x=new_data.x, y=new_data.y, edge_index=augmented_edge_index)


In [ ]:
# Tamaño del dataset creado
size_dataset = len(augmented_data)
print("Tamaño del dataset creado:", size_dataset)

# Cantidad de nodos, aristas y etiquetas en el dataset creado
num_nodes = augmented_data.num_nodes
num_edges = augmented_data.num_edges
num_labels = augmented_data.y.shape[0]
print("Cantidad de nodos:", num_nodes)
print("Cantidad de aristas:", num_edges)
print("Cantidad de etiquetas:", num_labels)

# Comparación con el dataset del paso 2
if num_nodes == new_data.num_nodes and num_labels == new_data.y.shape[0]:
    print("El dataset creado tiene la misma cantidad de nodos y etiquetas que el dataset del paso 2.")
else:
    print("El dataset creado no tiene la misma cantidad de nodos y etiquetas que el dataset del paso 2.")
if num_edges > new_data.num_edges:
    print("El dataset creado tiene más aristas que el dataset del paso 2.")

Tamaño del dataset creado: 3
Cantidad de nodos: 736389
Cantidad de aristas: 11825542
Cantidad de etiquetas: 736389
El dataset creado tiene la misma cantidad de nodos y etiquetas que el dataset del paso 2.
El dataset creado tiene más aristas que el dataset del paso 2.


#### Preguntas

1. ¿De qué tamaño queda el dataset creado?

R. El dataset creado tiene un tamaño de 3. Este número se refiere a la cantidad de elementos en el objeto Data (es decir, x, y y edge_index).

2. ¿Cuántos nodos, aristas y labels tiene?

R. Nodos: 736389
Aristas: 11825542
Etiquetas (labels): 736389

3. Comparando el nuevo dataset con el del paso 2., ¿Tenemos la misma cantidad de datos que el dataset original? ¿Por qué?

R. Comparando el nuevo dataset con el del paso 2:

La cantidad de nodos y etiquetas en el nuevo dataset es la misma que en el dataset del paso 2. Esto se debe a que no se añadió ningún nodo nuevo ni se cambiaron las etiquetas.
La cantidad de aristas en el nuevo dataset es mayor que en el dataset del paso 2. Esto se debe a que se agregaron nuevas aristas entre los papers escritos por el mismo autor, lo que aumentó la cantidad total de aristas en el grafo.
Por lo tanto, a pesar de que el nuevo dataset tiene la misma cantidad de nodos y etiquetas que el dataset del paso 2, contiene más información al tener más conexiones (aristas) entre los nodos. Esta información adicional puede ser muy útil para mejorar la eficacia de los modelos de aprendizaje automático basados en grafos.

## 4. Split del dataset


Carga el split precomputado desde el archivo `split`, nuevamente puedes descargar el archivo desde [este link](https://drive.google.com/file/d/1plLjfKhU7XcJshLvwZaoavdC_u8tL48z) o usar el comando que se encuentra a continuación.

In [ ]:
!gdown 1plLjfKhU7XcJshLvwZaoavdC_u8tL48z

Downloading...
From: https://drive.google.com/uc?id=1plLjfKhU7XcJshLvwZaoavdC_u8tL48z
To: /content/split
100% 5.89M/5.89M [00:00<00:00, 17.4MB/s]


In [ ]:
with open('drive/MyDrive//Backup/split', 'rb') as file:
  split_data = pickle.load(file)

print(split_data)

{'train': {'paper': tensor([     0,      1,      2,  ..., 736386, 736387, 736388])}, 'valid': {'paper': tensor([   332,    756,    784,  ..., 736364, 736367, 736370])}, 'test': {'paper': tensor([   359,    411,    608,  ..., 736358, 736384, 736385])}}


In [ ]:
print(type(split_data))
print(split_data.keys())
print(type(split_data['train']))
print(len(split_data['train']['paper']))
print(len(split_data['test']['paper']))
print(len(split_data['valid']['paper']))

<class 'dict'>
dict_keys(['train', 'valid', 'test'])
<class 'dict'>
629571
41939
64879


#### Preguntas

1. ¿En qué formato se encuentra guardado el split?

R.El split se encuentra guardado en formato de diccionario de Python, donde cada clave del diccionario representa una de las partes del split ('train', 'valid', y 'test') y su valor asociado es otro diccionario que contiene el tipo de nodo y un tensor con los índices de los nodos correspondientes a esa parte.

2. ¿En cuántas partes se dividirá el dataset?

R. El dataset se dividirá en tres partes: entrenamiento ('train'), validación ('valid') y prueba ('test').

3. ¿Qué porcentaje del dataset se asignó a cada parte?

R. Nodos asignados al entrenamiento: 629571 (85.49%)
Nodos asignados a la validación: 64879 (8.81%)
Nodos asignados a la prueba: 41939 (5.70%)

In [ ]:
# Verificar el tipo de objeto que se cargó
print("Tipo de objeto cargado:", type(split_data))

Tipo de objeto cargado: <class 'dict'>


In [ ]:
# Si el objeto es un diccionario, imprimir las claves
if isinstance(split_data, dict):
    print("Claves del diccionario:", split_data.keys())

Claves del diccionario: dict_keys(['train', 'valid', 'test'])


In [ ]:
with open('drive/MyDrive//Backup/split', 'rb') as file:
  split = pickle.load(file)

print(split)


{'train': {'paper': tensor([     0,      1,      2,  ..., 736386, 736387, 736388])}, 'valid': {'paper': tensor([   332,    756,    784,  ..., 736364, 736367, 736370])}, 'test': {'paper': tensor([   359,    411,    608,  ..., 736358, 736384, 736385])}}


In [ ]:
# Cantidad de nodos asignados a cada parte del split
train_nodes = len(split['train'])
valid_nodes = len(split['valid'])
test_nodes = len(split['test'])

# Porcentaje del dataset asignado a cada parte
total_nodes = train_nodes + valid_nodes + test_nodes
train_percentage = (train_nodes / total_nodes) * 100
valid_percentage = (valid_nodes / total_nodes) * 100
test_percentage = (test_nodes / total_nodes) * 100

print(f"Nodos asignados al entrenamiento: {train_nodes} ({train_percentage:.2f}%)")
print(f"Nodos asignados a la validación: {valid_nodes} ({valid_percentage:.2f}%)")
print(f"Nodos asignados a la prueba: {test_nodes} ({test_percentage:.2f}%)")


Nodos asignados al entrenamiento: 1 (33.33%)
Nodos asignados a la validación: 1 (33.33%)
Nodos asignados a la prueba: 1 (33.33%)


In [ ]:
print("Nodos de entrenamiento:", split['train'])
print("Nodos de validación:", split['valid'])
print("Nodos de prueba:", split['test'])


Nodos de entrenamiento: {'paper': tensor([     0,      1,      2,  ..., 736386, 736387, 736388])}
Nodos de validación: {'paper': tensor([   332,    756,    784,  ..., 736364, 736367, 736370])}
Nodos de prueba: {'paper': tensor([   359,    411,    608,  ..., 736358, 736384, 736385])}


In [ ]:
print("Tipo de los elementos en el conjunto de entrenamiento:", type(split['train'][0]))
print("Tipo de los elementos en el conjunto de validación:", type(split['valid'][0]))
print("Tipo de los elementos en el conjunto de prueba:", type(split['test'][0]))


KeyError: ignored

In [ ]:
# Cantidad de nodos asignados a cada parte del split
train_nodes = len(split['train']['paper'])
valid_nodes = len(split['valid']['paper'])
test_nodes = len(split['test']['paper'])

print(f"Nodos asignados al entrenamiento: {train_nodes} ({100 * train_nodes / num_nodes:.2f}%)")
print(f"Nodos asignados a la validación: {valid_nodes} ({100 * valid_nodes / num_nodes:.2f}%)")
print(f"Nodos asignados a la prueba: {test_nodes} ({100 * test_nodes / num_nodes:.2f}%)")


Nodos asignados al entrenamiento: 629571 (85.49%)
Nodos asignados a la validación: 64879 (8.81%)
Nodos asignados a la prueba: 41939 (5.70%)


In [ ]:
drive.flush_and_unmount()

## 5. Creacion de la red

### GCN

Crea una GCN, con `GCNConv`, que reciba como parámetro la cantidad de layer convolucionales internas que utilizará.

La GCN debe utilizar una función de activación ELu y un dropout entre cada layer; la probabilidad del dropout debe ser un parámetro de la red.

Cada capa debe normalizar los datos y las capas que debes crear son:

1. Una capa convolucional que tenga `in_channels` de entrada y `hidden_channels` de salida.
2. Un cantidad `hidden_layers` de capas convolucionales intermedias con `hidden_channels` de entrada y de salida.
3. Una capa convolucional que tenga `hidden_channels` de entrada y `out_channels` de salida.

Asegúrate que las capas internas sean parte del modelo.

In [ ]:
import torch
from torch.nn import ELU, Dropout
from torch_geometric.nn import GCNConv, BatchNorm

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_hidden_layers, dropout_prob):
        super(GCN, self).__init__()

        # Crear las capas convolucionales
        self.conv_in = GCNConv(in_channels, hidden_channels)
        self.conv_out = GCNConv(hidden_channels, out_channels)
        self.conv_hidden = torch.nn.ModuleList([GCNConv(hidden_channels, hidden_channels) for _ in range(num_hidden_layers)])

        # Crear las capas de batch normalization
        self.batch_norm_in = BatchNorm(hidden_channels)
        self.batch_norm_hidden = torch.nn.ModuleList([BatchNorm(hidden_channels) for _ in range(num_hidden_layers)])

        # Crear las capas de activación y dropout
        self.activation = ELU()
        self.dropout = Dropout(p=dropout_prob)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Capa de entrada
        x = self.conv_in(x, edge_index)
        x = self.batch_norm_in(x)
        x = self.activation(x)
        x = self.dropout(x)

        # Capas ocultas
        for conv, batch_norm in zip(self.conv_hidden, self.batch_norm_hidden):
            x = conv(x, edge_index)
            x = batch_norm(x)
            x = self.activation(x)
            x = self.dropout(x)

        # Capa de salida
        x = self.conv_out(x, edge_index)

        return x


Crea una instancia de tu modelo con 256 `hidden_channels`, 349 `out_channels`, 3 `hidden_layers` y una probabilidad de dropout de 0.3.

Imprime tu instancia del modelo.

In [ ]:
# Parámetros
in_channels = data.num_node_features  # Dependiendo de la representación de tus nodos, necesitarás ajustar esto.
hidden_channels = 256
out_channels = 349
num_hidden_layers = 3
dropout_prob = 0.3

# Crear la instancia del modelo
model = GCN(in_channels, hidden_channels, out_channels, num_hidden_layers, dropout_prob)

# Imprimir la instancia del modelo
print(model)


GCN(
  (conv_in): GCNConv(0, 256)
  (conv_out): GCNConv(256, 349)
  (conv_hidden): ModuleList(
    (0-2): 3 x GCNConv(256, 256)
  )
  (batch_norm_in): BatchNorm(256)
  (batch_norm_hidden): ModuleList(
    (0-2): 3 x BatchNorm(256)
  )
  (activation): ELU(alpha=1.0)
  (dropout): Dropout(p=0.3, inplace=False)
)


#### Preguntas

1. ¿Cuántas capas tiene, en total, el modelo?

R. En el modelo, hay un total de 5 capas convolucionales: una capa de entrada (conv_in), tres capas ocultas (conv_hidden) y una capa de salida (conv_out). Las capas de normalización por lotes y las funciones de activación y dropout, aunque son partes cruciales de la arquitectura de la red, a menudo no se cuentan como "capas" en el mismo sentido. Pero si consideramos "en total", el modelo tiene 9 "capas" incluyendo las capas de normalización por lotes. Y si además decidimos contar las funciones de activación y dropout como capas, entonces tendríamos un total de 11 capas








### Sage

Crea una GCN, con `SAGEConv`, que reciba como parámetro la cantidad de layers convolucionales internas que utilizará.

La GCN debe utilizar una función de activación ELu y un dropout entre cada layer; la probabilidad del dropout debe ser un parámetro de la red.

Debes incluír las siguientes capas:

1. Una capa convolucional que tenga `in_channels` de entrada y `hidden_channels` de salida.
2. Un cantidad `hidden_layers` de capas convolucionales intermedias con `hidden_channels` de entrada y de salida.
3. Una capa convolucional que tenga `hidden_channels` de entrada y `out_channels` de salida.

Instancia tu modelo con 256 `hidden_channels`, 349 `out_channels`, 4 `hidden_layers` y una probabilidad de dropout de 0.5

In [ ]:
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F

class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, hidden_layers, dropout):
        super(SAGE, self).__init__()

        # Primera capa convolucional
        self.conv_in = SAGEConv(in_channels, hidden_channels)

        # Capas convolucionales ocultas
        self.conv_hidden = torch.nn.ModuleList()
        for _ in range(hidden_layers):
            self.conv_hidden.append(SAGEConv(hidden_channels, hidden_channels))

        # Capa convolucional de salida
        self.conv_out = SAGEConv(hidden_channels, out_channels)

        # Función de activación ELU
        self.activation = torch.nn.ELU()

        # Dropout
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, x, edge_index):
        # Primera capa convolucional con activación y dropout
        x = self.conv_in(x, edge_index)
        x = self.activation(x)
        x = self.dropout(x)

        # Capas convolucionales ocultas con activación y dropout
        for conv in self.conv_hidden:
            x = conv(x, edge_index)
            x = self.activation(x)
            x = self.dropout(x)

        # Capa convolucional de salida
        x = self.conv_out(x, edge_index)

        return x


In [ ]:
model = SAGE(in_channels=256, hidden_channels=256, out_channels=349, hidden_layers=4, dropout=0.5)
print(model)


SAGE(
  (conv_in): SAGEConv(256, 256, aggr=mean)
  (conv_hidden): ModuleList(
    (0-3): 4 x SAGEConv(256, 256, aggr=mean)
  )
  (conv_out): SAGEConv(256, 349, aggr=mean)
  (activation): ELU(alpha=1.0)
  (dropout): Dropout(p=0.5, inplace=False)
)


#### Preguntas

1. ¿Cuántas capas tiene, en total, el modelo?

R. El modelo GraphSAGE tiene un total de 6 capas. Aquí está el desglose:

Una capa convolucional de entrada (conv_in): Esta capa convierte las características de entrada de 256 dimensiones a las características ocultas de 256 dimensiones.

Cuatro capas convolucionales ocultas (conv_hidden): Cada una de estas capas toma características de 256 dimensiones y las transforma en características de 256 dimensiones.

Una capa convolucional de salida (conv_out): Esta capa convierte las características ocultas de 256 dimensiones a las características de salida de 349 dimensiones.

Además, cada una de estas capas convolucionales es seguida por una función de activación ELU y un dropout. Sin embargo, estas no son consideradas capas en el sentido tradicional.

## 6. Entrenamiento de la red

Crea dos instancias de cada red y entrénalas utilizando los dos datsets creados anteriormente.

Puedes utilizar la función de activación, optimizador, capas ocultas y learning rate que prefieras, pero debes entrenar por al menos 100 épocas y asegurarte de que la red efectivamente esté aprendiendo (la función de pérdida vaya bajando al menos al inicio del entrenamiento).

**Importante:** El dataset cuenta con 349 clases distintas para la clasificación

In [ ]:
from torch_geometric.data import DataLoader


In [ ]:
model = GCN(in_channels=data.num_features, hidden_channels=256,
                out_channels=349, num_layers=2,
                dropout=0.5)

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout):
        super(GCN, self).__init__()
        self.conv_in = GCNConv(in_channels, hidden_channels)
        self.conv_hidden = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.conv_hidden.append(GCNConv(hidden_channels, hidden_channels))
        self.conv_out = GCNConv(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(p=dropout)
        self.activation = torch.nn.ELU()
        self.batch_norm_in = torch.nn.BatchNorm1d(hidden_channels)
        self.batch_norm_hidden = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.batch_norm_hidden.append(torch.nn.BatchNorm1d(hidden_channels))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv_in(x, edge_index)
        x = self.batch_norm_in(x)
        x = self.activation(x)
        x = self.dropout(x)
        for conv, batch_norm in zip(self.conv_hidden, self.batch_norm_hidden):
            x = conv(x, edge_index)
            x = batch_norm(x)
            x = self.activation(x)
            x = self.dropout(x)
        x = self.conv_out(x, edge_index)
        return F.log_softmax(x, dim=1)


In [ ]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout):
        super(SAGE, self).__init__()
        self.conv_in = SAGEConv(in_channels, hidden_channels)
        self.conv_hidden = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.conv_hidden.append(SAGEConv(hidden_channels, hidden_channels))
        self.conv_out = SAGEConv(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(p=dropout)
        self.activation = torch.nn.ELU()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv_in(x, edge_index)
        x = self.activation(x)
        x = self.dropout(x)
        for conv in self.conv_hidden:
            x = conv(x, edge_index)
            x = self.activation(x)
            x = self.dropout(x)
        x = self.conv_out(x, edge_index)
        return F.log_softmax(x, dim=1)


In [ ]:

# Definir una función de entrenamiento
def train(model, loader, optimizer):
    model.train()

    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Entrenar los modelos
for i in range(len(models)):
    # Asegúrate de usar el mismo optimizador y condiciones de entrenamiento
    optimizer = torch.optim.Adam(models[i].parameters(), lr=0.01)

    # Crear DataLoader para el dataset
    loader = DataLoader(datasets[i], batch_size=32, shuffle=True)

    for epoch in range(100):  # Número de épocas
        loss = train(models[i], loader, optimizer)
        # puedes agregar código aquí para registrar el rendimiento y hacer un seguimiento del aprendizaje
        print(f"Model {i+1}, Epoch {epoch+1}, Loss: {loss}")



# Crear las instancias de los modelos
gcn_model1 = GCN(in_channels=data.num_features, hidden_channels=256,
                out_channels=349, num_layers=2,
                dropout=0.5)

gcn_model2 = GCN(in_channels=data.num_features, hidden_channels=256,
                out_channels=349, num_layers=2,
                dropout=0.5)

sage_model1 = SAGE(in_channels=data.num_features, hidden_channels=256,
                   out_channels=349, num_layers=2,
                   dropout=0.5)

sage_model2 = SAGE(in_channels=data.num_features, hidden_channels=256,
                   out_channels=349, num_layers=2,
                   dropout=0.5)

# Lista de modelos y datasets para iterar a través de ellos
models = [gcn_model1, gcn_model2, sage_model1, sage_model2]
datasets = [original_dataset, augmented_dataset, original_dataset, augmented_dataset]

# Entrenar los modelos
for i in range(len(models)):
    # Asegurarte de usar el mismo optimizador y condiciones de entrenamiento
    optimizer = torch.optim.Adam(models[i].parameters(), lr=0.01)
    for epoch in range(100):  # Número de épocas
        train(models[i], datasets[i], split_dict['train'], optimizer)
        # puedes agregar código aquí para registrar el rendimiento y hacer un seguimiento del aprendizaje


NameError: ignored

### Preguntas

1. ¿Qué combinación de red-dataset obtiene mejor accuracy?
2. Comparando el dataset aumentado con el original, ¿Presenta alguna ventaja?, puedes considerar la velocidad de entrenamiento, el reultado final de la red o cualquier otro parámetro que creas significativo.
3. ¿El aumento en el dataset afecta de la misma manera a ambas redes? ¿Es significativa la diferencia?
4. Comparando los dos tipos de red, ¿Es significativa la diferencia entre los accuracy?

# Parte 2: Uso de batches en grafos

## 1. Carga del dataset

Usando `torch.geometric`, carga el dataset `Mutagenicity` de `TUDataset`

In [ ]:
from torch_geometric.datasets import TUDataset

# Cargar el dataset Mutagenicity
dataset = TUDataset(root='/tmp/Mutagenicity', name='Mutagenicity')

print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')


Extracting /tmp/Mutagenicity/Mutagenicity/Mutagenicity.zip
Processing...


Dataset: Mutagenicity(4337):
Number of graphs: 4337
Number of features: 14
Number of classes: 2


Done!


#### Preguntas

Una vez cargado el dataset debes responder las siguientes preguntas:

1. ¿De qué tipo de objeto es el dataset cargado?

R. El dataset cargado es de tipo torch_geometric.datasets.tu_dataset.TUDataset.

2. ¿Cuántos grafos tiene el dataset?

R. El dataset contiene 4337 grafos.

3. ¿Cuántos nodos y aristas tiene, en promedio, cada grafo?

R. En promedio, cada grafo en el conjunto de datos tiene aproximadamente 30.32 nodos y 61.54 aristas.

4. Elige un grafo cualquiera del dataset (puedes ser el primero) y responde:
    1. ¿Cuántos nodos y aristas tiene el grafo?

    R. El grafo tiene 16 nodos y 32 aristas.

    2. ¿El grafo tiene self loops?

    R. El grafo no tiene self-loops. Un self-loop es una arista que se conecta a un nodo consigo mismo.

    3. ¿El grafo es dirigido?

    R. El grafo no es dirigido. En un grafo dirigido, las aristas tienen una dirección asociada (van de un nodo a otro). En cambio, en este caso, las aristas no tienen una dirección, lo que significa que la conexión entre dos nodos es bidireccional.

    4. ¿El grafo tiene nodos aislados?
    
    R. El grafo no tiene nodos aislados. Un nodo aislado es un nodo que no tiene aristas conectadas a él


In [ ]:
# Tipo de objeto del dataset
print(f"Tipo de objeto del dataset: {type(dataset)}")

# Cantidad de grafos en el dataset
print(f"Cantidad de grafos en el dataset: {len(dataset)}")

# Promedio de nodos y aristas en los grafos del dataset
num_nodes = [data.num_nodes for data in dataset]
num_edges = [data.num_edges for data in dataset]
avg_nodes = sum(num_nodes) / len(dataset)
avg_edges = sum(num_edges) / len(dataset)

print(f"Número promedio de nodos por grafo: {avg_nodes}")
print(f"Número promedio de aristas por grafo: {avg_edges}")

# Selecciona un grafo del dataset (p. ej., el primero)
data = dataset[0]

# Características del grafo seleccionado
print(f"\nPara el primer grafo del dataset:")
print(f"Número de nodos: {data.num_nodes}")
print(f"Número de aristas: {data.num_edges}")
print(f"El grafo tiene self loops: {'Sí' if data.contains_self_loops() else 'No'}")
print(f"El grafo es dirigido: {'Sí' if data.is_directed() else 'No'}")
print(f"El grafo tiene nodos aislados: {'Sí' if data.contains_isolated_nodes() else 'No'}")


Tipo de objeto del dataset: <class 'torch_geometric.datasets.tu_dataset.TUDataset'>
Cantidad de grafos en el dataset: 4337
Número promedio de nodos por grafo: 30.317731150564907
Número promedio de aristas por grafo: 61.53885174083468

Para el primer grafo del dataset:
Número de nodos: 16
Número de aristas: 32
El grafo tiene self loops: No
El grafo es dirigido: No
El grafo tiene nodos aislados: No


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'contains_self_loops' is deprecated, use 'has_self_loops' instead
  warnings.warn(out)
/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)


## 2. Split del dataset

Divide aleatoriamente el dataset entre 3 sets, entrenamiento, validación y test. Asigna un 80%, 10% y 10% de los datos a cada set, respectivamente.

Intenta que la distribución quede relativamente uniforme respecto al tamaño promedio de cada set.

In [ ]:
import torch

train_size = int(0.8 * len(dataset))
valid_size = (len(dataset) - train_size) // 2
test_size = len(dataset) - train_size - valid_size

torch.manual_seed(0)  # Para que el split sea reproducible
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(valid_dataset)}")
print(f"Testing dataset size: {len(test_dataset)}")


Training dataset size: 3469
Validation dataset size: 434
Testing dataset size: 434


#### Preguntas

1. ¿Cuántos grafos tiene cada dataset?

R. Cada dataset tiene la siguiente cantidad de grafos:

El dataset de entrenamiento tiene 3469 grafos.
El dataset de validación tiene 434 grafos.
El dataset de prueba tiene 434 grafos.

2. ¿Cuántos nodos y aristas tiene, en promedio, cada grafo?

R. Cada grafo en los datasets tiene, en promedio, la siguiente cantidad de nodos y aristas:

En el dataset de entrenamiento, cada grafo tiene en promedio 30.02 nodos y 61.19 aristas.
En el dataset de validación, cada grafo tiene en promedio 29.63 nodos y 60.63 aristas.
En el dataset de prueba, cada grafo tiene en promedio 33.39 nodos y 65.25 aristas.




In [ ]:
# Cantidad de grafos en cada dataset
print(f"Number of graphs in the training dataset: {len(train_dataset)}")
print(f"Number of graphs in the validation dataset: {len(valid_dataset)}")
print(f"Number of graphs in the test dataset: {len(test_dataset)}")

# Cálculo del número promedio de nodos y aristas en cada grafo
def calculate_average_nodes_edges(graph_dataset):
    total_nodes = 0
    total_edges = 0
    for data in graph_dataset:
        total_nodes += data.num_nodes
        total_edges += data.num_edges
    avg_nodes = total_nodes / len(graph_dataset)
    avg_edges = total_edges / len(graph_dataset)
    return avg_nodes, avg_edges

avg_nodes_train, avg_edges_train = calculate_average_nodes_edges(train_dataset)
avg_nodes_valid, avg_edges_valid = calculate_average_nodes_edges(valid_dataset)
avg_nodes_test, avg_edges_test = calculate_average_nodes_edges(test_dataset)

print(f"Average number of nodes in the training dataset: {avg_nodes_train}")
print(f"Average number of edges in the training dataset: {avg_edges_train}")
print(f"Average number of nodes in the validation dataset: {avg_nodes_valid}")
print(f"Average number of edges in the validation dataset: {avg_edges_valid}")
print(f"Average number of nodes in the test dataset: {avg_nodes_test}")
print(f"Average number of edges in the test dataset: {avg_edges_test}")


Number of graphs in the training dataset: 3469
Number of graphs in the validation dataset: 434
Number of graphs in the test dataset: 434
Average number of nodes in the training dataset: 30.01931392332084
Average number of edges in the training dataset: 61.188238685500146
Average number of nodes in the validation dataset: 29.629032258064516
Average number of edges in the validation dataset: 60.62672811059908
Average number of nodes in the test dataset: 33.39170506912443
Average number of edges in the test dataset: 65.25345622119816


## 3. Creación de los dataloaders


Crea un `Dataloader` para cada dataset. Elige el número de batches de cada uno según creas conveniente.

In [ ]:
from torch_geometric.data import DataLoader

# Decide on the batch size. This may depend on the memory available on your machine.
# A typical choice of batch size is between 32 and 128.
batch_size = 32

# Create a DataLoader for each dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


#### Preguntas

1. ¿Qué tamaños de batch utilizaste para cada set? Justifica el por qué elegiste cada uno de ellos y qué ventajas/desventajas tiene comparado con otros valores.

R. El tamaño de los batches para cada set es 32. Este tamaño se elige comúnmente en las aplicaciones de aprendizaje automático por ser un número potencia de 2, lo cual resulta beneficioso para la optimización de la GPU. También ofrece un buen equilibrio entre la eficiencia del aprendizaje y los requisitos de memoria.

Sin embargo, la elección del tamaño de batch puede depender de varios factores, como la memoria disponible en tu hardware, el tipo de modelo que se está entrenando, y el tamaño del conjunto de datos. Un tamaño de batch más grande puede llevar a un aprendizaje más rápido (menos iteraciones por época), pero también puede conducir a estimaciones de gradiente menos precisas y requerir más memoria. Por el contrario, un tamaño de batch más pequeño puede llevar a estimaciones de gradiente más precisas, pero puede requerir más iteraciones por época y ser menos eficiente en términos de computación.

2. ¿Cuántos batches tiene cada uno de los sets?

R. podemos ver que el conjunto de entrenamiento se divide en 109 batches, mientras que los conjuntos de validación y prueba se dividen en 14 batches cada uno.

3. ¿De qué tamaño es cada batch resultante?

R. Cada batch tiene un tamaño de 32, aunque el tamaño del último batch puede ser menor si el tamaño total del conjunto de datos no es un múltiplo de 32.

In [ ]:
# número de batches en el conjunto de entrenamiento
num_batches_train = len(train_loader)

# número de batches en el conjunto de validación
num_batches_valid = len(valid_loader)

# número de batches en el conjunto de prueba
num_batches_test = len(test_loader)

print(f"Número de batches en el conjunto de entrenamiento: {num_batches_train}")
print(f"Número de batches en el conjunto de validación: {num_batches_valid}")
print(f"Número de batches en el conjunto de prueba: {num_batches_test}")


Número de batches en el conjunto de entrenamiento: 109
Número de batches en el conjunto de validación: 14
Número de batches en el conjunto de prueba: 14


In [ ]:
# Ver el tamaño del primer batch en el conjunto de entrenamiento
first_batch = next(iter(train_loader))
print(f"El tamaño del primer batch en el conjunto de entrenamiento es: {len(first_batch)}")


El tamaño del primer batch en el conjunto de entrenamiento es: 32


## 4. Creacion de la red

Crea una GCN, con `GCNConv`, que reciba como parámetro la cantidad de layers convolucionales que utilizará para crear los embeddings de cada grafo.

La GCN debe utilizar una función de activación ReLu.

Debe tener las siguientes capas:

1. Una capa convolucional que tenga `in_channels` de entrada y `hidden_channels` de salida.
2. Un cantidad `hidden_layers` de capas intermedias con `hidden_channels` de entrada y de salida.
3. Un _mean pool_ que obtenga el mebedding del grafo a partir de los embeddings de los nodos.
4. Un dropout antes de la última capa. La probabilidad del dropout debe ser un parámetro de la red.
4. Una capa *fully connected* que tenga `hidden_channels` de entrada y `out_channels` de salida.

Asegúrate que las capas internas sean parte del modelo.

In [ ]:
import torch
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout):
        super(GCN, self).__init__()
        self.conv_in = GCNConv(in_channels, hidden_channels)
        self.conv_hidden = torch.nn.ModuleList([GCNConv(hidden_channels, hidden_channels) for _ in range(num_layers - 1)])
        self.dropout = torch.nn.Dropout(dropout)
        self.fc_out = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Primera capa convolucional
        x = self.conv_in(x, edge_index)
        x = F.relu(x)

        # Capas convolucionales intermedias
        for conv in self.conv_hidden:
            x = conv(x, edge_index)
            x = F.relu(x)

        # Pooling de los embeddings de los nodos para obtener el embedding del grafo
        x = global_mean_pool(x, batch)

        # Dropout y última capa fully connected
        x = self.dropout(x)
        x = self.fc_out(x)

        return x


Crea una instancia de tu modelo con `hidden_layers = 2` e imprime los detalles del modelo

In [ ]:
model = GCN(in_channels=dataset.num_node_features, hidden_channels=64, out_channels=dataset.num_classes, num_layers=2, dropout=0.5)


In [ ]:
print(model)


GCN(
  (conv_in): GCNConv(14, 64)
  (conv_hidden): ModuleList(
    (0): GCNConv(64, 64)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_out): Linear(in_features=64, out_features=2, bias=True)
)


#### Preguntas

1. ¿Cuántas capas tiene el modelo?

R. El modelo tiene 5 capas:

La primera capa es una capa de convolución gráfica (GCNConv) que transforma las características de entrada a características ocultas.

Luego tienes un número específico de capas ocultas (en este caso 2) que son capas de convolución gráfica que transforman las características ocultas en nuevas características ocultas. Por lo tanto, cada capa de estas cuenta como una capa adicional.

Una capa de dropout que regulariza el modelo durante el entrenamiento, ayudando a prevenir el sobreajuste.

Una capa de agrupación global (global mean pooling), que calcula el promedio de las características de los nodos para obtener una representación de todo el grafo.

Finalmente, tienes una capa totalmente conectada (linear), que transforma las características del grafo (después de la agrupación) en la predicción de salida.

Por lo tanto, en total, tu modelo tiene 5 capas.

## 5. Entrenamiento de la red

Crea tres instancias del a red, usando `hidden_layers = 0`, `hidden_layers = 2` y `hidden_layers = 4`.

Entrena cada red por al menos 100 épocas, considerando el set de validación en el proceso. Los hiperparámetros, función de pérdida y optimizador quedan a tu criterio, pero debes asegurarte que la red aprende (la función de pérdida va bajando, al menos en las primeras etapas de entrenamiento).

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

# Configuración de la pérdida y el optimizador
criterion = CrossEntropyLoss()
learning_rate = 0.01

# Lista para almacenar los modelos
models = []

# Crear las instancias de los modelos y entrenarlos
for hidden_layers in [0, 2, 4]:
    model = GCN(in_channels=dataset.num_features, hidden_channels=256,
                out_channels=dataset.num_classes, num_layers=hidden_layers,
                dropout=0.5)
    models.append(model)

    # Configurar el optimizador
    optimizer = Adam(model.parameters(), lr=learning_rate)

    # Entrenar el modelo
    for epoch in range(100):
        for batch in train_loader:
            optimizer.zero_grad()
            out = model(batch)
            loss = criterion(out, batch.y)
            loss.backward()
            optimizer.step()


#### Preguntas

Una vez cargado el dataset debes responder las siguientes preguntas:

1. En términos teóricos, ¿Qué representa la cantidad de `hidden_layers` del modelo?

R. En términos teóricos, la cantidad de hidden_layers en un modelo de red neuronal representa la profundidad del modelo. En una Graph Convolutional Network (GCN), cada capa oculta puede considerarse como un paso de propagación de mensajes a través del grafo, donde los nodos actualizan sus características basándose en la información de sus vecinos

2. En términos teóricos, ¿Qué diferencias genera instanciar la red con 0, 2 y 4 `hidden_layers`?

R. En términos teóricos, al instanciar una red con 0 hidden_layers se está creando una red muy sencilla que proyecta las características de entrada a las de salida, sin realizar ninguna transformación o aprendizaje intermedio. Con 2 hidden_layers, la red tiene la oportunidad de aprender representaciones más complejas, ya que hay capas intermedias que pueden capturar características no lineales. Con 4 hidden_layers, la red es aún más profunda, y por lo tanto tiene una mayor capacidad para aprender representaciones más abstractas y complejas. Sin embargo, también corre un mayor riesgo de sobreajuste, ya que más capas pueden llevar a que el modelo memorice los datos de entrenamiento en lugar de aprender patrones generalizables.

3. ¿Qué red tiene mejor accuracy? ¿A qué crees que se debe esta diferencia?

R. La precisión de cada red dependerá de varios factores, incluyendo el número de épocas de entrenamiento, el tamaño de los batches, la tasa de aprendizaje, y otros hiperparámetros. En general, se podría esperar que una red con un número moderado de capas ocultas (por ejemplo, 2) tenga una mejor precisión que una red con 0 o muchas capas ocultas, porque tiene la capacidad de aprender representaciones más complejas sin caer en el sobreajuste. Sin embargo, esto también dependerá del conjunto de datos específico y la tarea de clasificación.

4. Crea una nueva instancia de la red con un número de `hidden_layers` distinto al utilizado. Compara los resultados obtenidos y justifica por qué crees que son esos, considerando la cantidad de hidden layers que elegiste.
